# **PREPARE DATA**

In [1]:
import pandas as pd
import pickle
import numpy as np
import glob
import os 
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.linear_model import SGDClassifier


import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ZheniNikolaevaDimitr\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
trainig_list = glob.glob(os.path.join(os.getcwd(), "./data/train-mails", "*.txt"))
test_data = glob.glob(os.path.join(os.getcwd(), "./data/test-mails", "*.txt")) #with those 2 we get the training and testing data from the folders

training_data =  pd.DataFrame(columns=['text', 'spam'])
test_data =  pd.DataFrame(columns=['text', 'spam']) #create dataframe - creating columns text and spam

for index, file_path in enumerate(trainig_list):
    with open(file_path) as f_input:
        spam = 0
        if 'spmsg' in file_path:
            spam = 1
        training_data.loc[index] = [f_input.read(), spam] #parse the data from the training list and see whether its a spam or not based on spmmsg tag in title
    
for file_path in trainig_list:
    with open(file_path) as f_input:
        spam = 0
        if 'spmsg' in file_path:
            spam = 1
        test_data.loc[index] = [f_input.read(), spam] #does the same? ask toms
 

In [3]:
training_data.shape #it will give the answer , show data as, as rows and columns , it makes it like that by default, with this you can see how the data looks like
training_data.head(3) # if i put 5 it will show 5 outputs

,text,spam
0,Subject: re : 2 . 882 s - > np np\n\n> deat : ...,0
1,Subject: s - > np + np\n\ndiscussion s - > np ...,0
2,Subject: 2 . 882 s - > np np\n\n. . . 's much ...,0


In [4]:
def preprocessor(text):
    # remove HTML markup
    text = re.sub('<[^/>]*>', ' ', text)
    # remove no-words
    text = re.sub('[\W]', ' ', text)
    # convert all to lower case
    text = text.lower()
    text = text.split()
    # remove stop words
    stop = stopwords.words('english')
    porter = PorterStemmer() #to get rid of the multiple similar words, get the base of the word == run running ran / for that it will give run
    text = [porter.stem(word) for word in text if not word in stop]
    text = ' '.join(text)
    
    return text; #this method cleans the data from prepinatelni znaci and useless words = and or the


training_data['text'] = training_data['text'].apply(preprocessor) # apply the above method on the training data
training_data['text']#displaying the result of what we did up there in this cell, down we have a display of the cleaned training data

0      subject 2 882 np np deat sun 15 dec 91 2 25 2 ...
1      subject np np discuss np np remind ago read so...
2      subject 2 882 np np much restrict np np np pro...
3      subject gent confer listserv intern confer 199...
4      subject queri caus korean anyon point book art...
5      subject l2 learn cultur empathi graduat studen...
6      subject psycholinguist teach undergradu cours ...
7      subject german corpora line corpu modern germa...
8      subject hi help design experi mandarin tone pa...
9      subject job univers utah linguist program univ...
10     subject job montreal post de professeur e regu...
11     subject lectureship linguist c h l f e n g l h...
12     subject neg concord interest grammar neg conco...
13     subject neg concord tri compil list commerci a...
14     subject queri email address doe anybodi email ...
15     subject post hi work phonet project modern iri...
16     subject queri iceland anyon tell anyth whether...
17     subject search doe anyon

# Creating a word dictionary

In [5]:
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]    
    all_words = []       
    for mail in emails:    
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:  #Body of email is only 3rd line of text file
                    words = line.split()
                    all_words += words # with this method we create a dictionary, which is a matrix that will show how many times each word was repeated in one email
    
    dictionary = Counter(all_words)
    list_to_remove = dictionary.keys()
    for item in list(list_to_remove):
        if item.isalpha() == False: 
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(3000)
    return dictionary # we make the list to remove the words that are repeated once or short words = look up the tutirial

# Feature extraction process.

In [6]:
def extract_features(mail_dir): 
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000)) # we make another matrix with 3000 columns / see on tutorial / to extract the features from the directory
    docID = 0;
    for fil in files:
      with open(fil) as fi:
        for i,line in enumerate(fi): #looping throug the lines of the file ; enumerate might meand turn text datat into nummeric
          if i == 2: #skiping first 2 lines of email cause its cc and empty line
            words = line.split() # creating array of words from a line of the file 
            for word in words:
              wordID = 0
              for i,d in enumerate(dictionary): 
                if d[0] == word:
                  wordID = i
                  features_matrix[docID,wordID] = words.count(word)
        docID = docID + 1     
    return features_matrix # this matrix has word and emails / it is a representation of how many times a selected word has beeen used in each of the provided mail

# Training the classifiers.

** List of imports for classifiers **

In [7]:
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import HashingVectorizer
import re #importing studff



** Create a dictionary of words with its frequency **

In [8]:
train_dir = './data/train-mails' #we target the training directory
dictionary = make_Dictionary(train_dir)  # we creati a dictionary from the taining directory

** Prepare feature vectors per training mail and its labels **

In [9]:
train_labels = np.zeros(702)# creates an empty array or dataset
train_labels[351:701] = 1 
train_matrix = extract_features(train_dir) #creates training  matrix using extract features function
print(train_matrix) 

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


** Training SVM and Naive bayes classifier **

In [10]:
MyNB = MultinomialNB()
MySVC = LinearSVC()
MySGDC = SGDClassifier(loss='log', random_state=1, max_iter=1) #first 3 lines is creating the classifiers
MyNB.fit(train_matrix,train_labels)
MySVC.fit(train_matrix,train_labels)
MySGDC.fit(train_matrix,train_labels)#last 3 lines is training the classifiers

C:\Users\ZheniNikolaevaDimitr\Anaconda2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=1, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

** Test the unseen mails for Spam **

In [11]:
test_dir = './data/test-mails' #set test directory
test_matrix = extract_features(test_dir) #extract features using the method from above in order to test the data
test_labels = np.zeros(260) #creating an empty array with 260 elements
test_labels[130:260] = 1 


result_MyNB = MyNB.predict(test_matrix)
print ("MultinomialNB: Accuracy is ", accuracy_score(test_labels, result_MyNB)*100)

result_MySVC = MySVC.predict(test_matrix)
print ("SVC: Accuracy is ", accuracy_score(test_labels, result_MySVC)*100)

result_MySGDC = MySGDC.predict(test_matrix)
print ("SGDC: Accuracy is ", accuracy_score(test_labels, result_MySGDC)*100) # create predictions and print them to see which algorithm is the most accurate

# Create the confusion matrix fot Mayes and SVC
confusion_mat_NB = confusion_matrix(test_labels,result_MyNB)
confusion_mat_SVC = confusion_matrix(test_labels,result_MySVC)
confusion_mat_SGDC = confusion_matrix(test_labels,result_MySGDC)

('MultinomialNB: Accuracy is ', 96.15384615384616)
('SVC: Accuracy is ', 96.15384615384616)
('SGDC: Accuracy is ', 93.07692307692308)


** Visualize confusion matrix **

In [12]:
# Visualize confusion matrix for bayes
plt.imshow(confusion_mat_NB, interpolation='nearest')
plt.title('Confusion matrix: MultinomialNB')
plt.colorbar()
ticks = np.arange(2)
plt.xticks(ticks, ticks)
plt.yticks(ticks, ticks)
plt.ylabel('True labels')
plt.xlabel('Predicted labels')
plt.show() 
#we have true and predicted labels  ,read on her slides 
#thw numbers will be displayed underneath and are showing numbers corresponding to the color scale and based on those numbers tha colots are being computed

AttributeError: 'module' object has no attribute 'to_rgba'

In [ ]:
# Visualize confusion matrix for SVC
plt.imshow(confusion_mat_SVC, interpolation='nearest')
plt.title('Confusion matrix: SVC')
plt.colorbar()
ticks = np.arange(2)
plt.xticks(ticks, ticks)
plt.yticks(ticks, ticks)
plt.ylabel('True labels')
plt.xlabel('Predicted labels')
plt.show()

In [ ]:
# Visualize confusion matrix for SGDC
plt.imshow(confusion_mat_SGDC, interpolation='nearest')
plt.title('Confusion matrix: SGDC')
plt.colorbar()
ticks = np.arange(2)
plt.xticks(ticks, ticks)
plt.yticks(ticks, ticks)
plt.ylabel('True labels')
plt.xlabel('Predicted labels')
plt.show()

In [ ]:
print("MultinomialNB: ")
print(confusion_matrix(result_MyNB, test_labels))
print("SVC: ")
print(confusion_matrix(result_MySVC, test_labels))
print("SGDC: ")
print(confusion_matrix(result_MySGDC, test_labels))

# here we get the numbers for each confusion matrix that are determinig the color

** Prediction for Bayes **

In [ ]:
# Validate the accuracy of the predictions for Bayes
predictionsNB = MyNB.predict(test_matrix)
print('Accuracy score: ', format(accuracy_score(test_labels, predictionsNB)))
print('Precision score: ', format(precision_score(test_labels, predictionsNB)))
print('Recall score: ', format(recall_score(test_labels, predictionsNB)))
print('F1 score: ', format(f1_score(test_labels, predictionsNB))) # printing all these out to validate the prediction

In [ ]:
# Validate the accuracy of the predictions
predictionsSVC = MySVC.predict(test_matrix)
print('Accuracy score: ', format(accuracy_score(test_labels, predictionsSVC)))
print('Precision score: ', format(precision_score(test_labels, predictionsSVC)))
print('Recall score: ', format(recall_score(test_labels, predictionsSVC)))
print('F1 score: ', format(f1_score(test_labels, predictionsSVC))) # printing all these out to validate the prediction

In [ ]:
# Validate the accuracy of the predictions
predictionsSGDC = MySGDC.predict(test_matrix)
print('Accuracy score: ', format(accuracy_score(test_labels, predictionsSGDC)))
print('Precision score: ', format(precision_score(test_labels, predictionsSGDC)))
print('Recall score: ', format(recall_score(test_labels, predictionsSGDC)))
print('F1 score: ', format(f1_score(test_labels, predictionsSGDC))) # printing all these out to validate the prediction

# Export the model

In [ ]:
# create dir and subdir for pickled objects (export of the built model)
dest = os.path.join('model', 'pickles')# create a directory where to save our model
if not os.path.exists(dest):#if it doesnt exist
    os.makedirs(dest)#create a new one

In [ ]:
# serialize the model
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)
pickle.dump(MyNB, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)
# we serialize it in order to save it 

# Load the model

In [ ]:
cur_dir = os.getcwd() # get current directory
stop = pickle.load(open(
                os.path.join('model', 
                'pickles', 
                'stopwords.pkl'), 'rb')) #we load the stopwords 



classifier = pickle.load(open(
                os.path.join('model', 
                'pickles', 
                'classifier.pkl'), 'rb'))    # we load the classifier   

In [ ]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer) # it is tokanizing stuff 

In [ ]:
# reuse the model in order ot make a new prediction
label = {0:'ham', 1:'spam'} # 0 is ham and 1 is spam, ham means teh mail is not spam

# predict() returns predicted label
# predict_proba(X) returns probability
example_dir = './data/example' #we make a new directory with only 1 file
X = extract_features(example_dir)#we extract the features in that X directory in order to make a prediction based on the features

print('Prediction 1: %s\nProbability 1: %.2f%%' %(label[classifier.predict(X)[0]], np.max(classifier.predict_proba(X))*100))
#print(np.max(classifier.predict_proba(X)))


#example2 = ['Last chance to save! We are extending our Cyber Monday Sale through the end of today. You won’t see another deal like this until next year, so act now for 30% off annual team accounts.']
#X = vect.transform(example2)
#print('Prediction 2: %s\nProbability 2: %.2f%%' %(label[classifier.predict(X)[0]], np.max(classifier.predict_proba(X))*100))

# Develop the Web App

In [ ]:
%%writefile mywebapp.py
from collections import Counter
import numpy as np
import re
import pickle
import os #imports
from flask import Flask, request, session, g, redirect, url_for, abort, \
     render_template, flash, jsonify
app = Flask(__name__, template_folder='./templates') #initialize the flask - used to create web apps with python- and we set the templates foldet for the template of the html

label = {0:'ham', 1:'spam'}
stop = pickle.load(open(
                os.path.join('model', 
                'pickles', 
                'stopwords.pkl'), 'rb')) # loading the stopwords

classifier = pickle.load(open(
                os.path.join('model', 
                'pickles', 
                'classifier.pkl'), 'rb')) # loading the classifier

def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]    
    all_words = []       
    for mail in emails:    
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:  #Body of email is only 3rd line of text file
                    words = line.split()
                    all_words += words 
    
    dictionary = Counter(all_words)
    list_to_remove = dictionary.keys()
    for item in list(list_to_remove):
        if item.isalpha() == False: 
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(3000)
    return dictionary #same function to create a dictionary - we put it here in order for pythn to be able to use it 


def extract_features(mail_dir): 
    train_dir = './data/train-mails'
    dictionary = make_Dictionary(train_dir)
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000))
    docID = 0;
    for fil in files:
      with open(fil) as fi:
        for i,line in enumerate(fi):
          if i == 2:
            words = line.split()
            for word in words:
              wordID = 0
              for i,d in enumerate(dictionary):
                if d[0] == word:
                  wordID = i
                  features_matrix[docID,wordID] = words.count(word)
        docID = docID + 1     
    return features_matrix # same extract feature because this is a python file





 
@app.route("/")
def index():
  return render_template('index.html') # set the route

def write_mail_to_file(mailContent = ''):
  path = './data/example'
  filename = 'example.txt'
  if not os.path.exists(path):
      os.makedirs(path) 

  f = open(os.path.join(path, filename), 'w+')
  f.write(mailContent)
  f.close() #function tp save the mail tht we he have inputted to save it in the file exapmle . txt in the folder data/example

@app.route("/check-email", methods=['POST'])
def check_mail():
  try:
      print(request.form)
      write_mail_to_file(request.form['mailContent'])
      example_dir = './data/example'
      X = extract_features(example_dir)

      print('Prediction 1: %s\nProbability 1: %.2f%%' %(label[classifier.predict(X)[0]], np.max(classifier.predict_proba(X))*100))

      res = {"success": True, "isSpam": label[classifier.predict(X)[0]], "probability": np.max(classifier.predict_proba(X))*100}
      return jsonify(res)
  except Exception as e:
        print(e)
        res = {"success": False, "error": e}
        return jsonify(res)  # post method to make the request based on what you have and return result wether its a spam or not and return the probabilit of it being true


if __name__ == "__main__":
  app.run(port = 5002, debug=True) # this is for flask so it knows on which port python flask app should run the code


In [ ]:

import subprocess as sub

# this opens a windows terminal
sub.call('start /wait python mywebapp.py', shell=True) #run the app and open the terminal sp we can see what errors we get
